In [182]:
import sys, os, pickle
from keras.models import Sequential
import pandas as pd
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import BatchNormalization
from keras.optimizers import Adam, RMSprop, SGD
import sys, re, glob
import numpy as np
from keras.layers import Input, Dense, Flatten
import sys, re, glob
import pandas as pd
import numpy as np
from lxml import etree
from sklearn.model_selection import StratifiedKFold 

In [117]:
def mac_to_id(mac):
    return int(mac.split(':')[-1], 16) - 1

def ip_to_id(ip):
    return int(ip.split('.')[-1]) - 1

def entropy(arr):
    s = arr.sum()
    arr = arr / s
    arr = arr * np.log(arr) / np.log(2)
    return -(arr.sum() / np.log(len(arr)))

In [4]:
input_file = "/home/nsuprotivniy/Desktop/output"

In [394]:
for i in range(100):
    os.rename('../data/ddos/{}_output/'.format(i), '../data/ddos/output_{}/'.format(i))

array([ 1.16229991e+00,  2.00000000e+00,  2.12000000e+02,  4.12962963e+01,
        2.70683191e+03,  1.12465369e+00,  2.00000000e+00,  2.37000000e+02,
        4.12962963e+01,  3.17960114e+03, -2.31000000e+02,  8.50000000e+01,
        0.00000000e+00,  2.85346154e+03,  1.42396524e+00,  1.00000000e+00,
        3.17000000e+02,  2.32000000e+02,  2.85346154e+03,  4.60000000e+02,
        2.20000000e+02])

In [176]:
diff

array([[-231.        ],
       [  85.        ],
       [   0.        ],
       [2853.46153846]])

In [343]:
proto_dict = {
    'ns3::WifiMacHeader' : 'MAC',
    'ns3::LlcSnapHeader' : 'LLC',
    'ns3::ArpHeader' : 'ARP',
    'ns3::Ipv4Header' : 'IPv4',
    'ns3::UdpHeader' : 'UDP',
    'ns3::aodv::TypeHeader' : 'AODV_Type',
    'ns3::aodv::RrepHeader' : 'AODV_RREP',
    'ns3::aodv::RreqHeader' : 'AODV_RREQ',
    'ns3::aodv::RrepAckHeader' : 'AODV_RACK',
    'ns3::aodv::RerrHeader' : 'AODV_RERR',
    'ns3::Icmpv4Header' : 'ICMPv4_HEADER',
    'ns3::Icmpv4TimeExceeded' : 'ICMPv4_TE',
    'ns3::Icmpv4DestinationUnreachable' : 'ICMPv4_DU'
}

def parce_packets(input_file):
    with open(input_file, 'r') as f:
        data = f.read()
    data = data.split('meta-info="')
    meta_info = []
    data.pop(0)
    for b in data:
        meta_info.append(b.split('"')[0])
    packets = []
    for m in meta_info:
        raw = {}
        for proto in proto_dict:
            if m.find(proto) != -1:
                raw[proto_dict[proto]] = m.split(proto+' (')[1].split(') ns3')[0]
        p = {}
        if 'MAC' in raw and 'LLC' in raw:
            p['MAC'] = {}
            p['MAC']['src'] = raw['MAC'].split('SA=')[1].split(',')[0]
            p['MAC']['dst'] = raw['MAC'].split('DA=')[1].split(',')[0]
            p['MAC']['type'] = int(raw['LLC'].split('type ')[1], 16)
        if 'ARP' in raw:
            p['ARP'] = {}
            if raw['ARP'].find('request') != -1:
                p['ARP']['type'] = 'request'
            else:
                p['ARP']['type'] = 'reply'
                p['ARP']['MAC_dst'] = raw['ARP'].split('dest mac: ')[1].split(' ')[0]
            p['ARP']['MAC_src'] = raw['ARP'].split('source mac: ')[1].split(' ')[0]
            p['ARP']['IP_src'] = raw['ARP'].split('source ipv4: ')[1].split(' ')[0]
            p['ARP']['IP_dst'] = raw['ARP'].split('dest ipv4: ')[1]
        if 'IPv4' in raw:
            p['IPv4'] = {}
            ipv4data = raw['IPv4'].split(' ')
            p['IPv4']['src'] = ipv4data[-3]
            p['IPv4']['dst'] = ipv4data[-1]
            p['IPv4']['proto'] = int(raw['IPv4'].split('protocol ')[1].split(' ')[0])
            p['IPv4']['ttl'] = int(raw['IPv4'].split('ttl ')[1].split(' ')[0])
            p['IPv4']['length'] = int(raw['IPv4'].split('length: ')[1].split(' ')[0])
        if 'UDP' in raw:
            p['UDP'] = {}
            p['UDP']['length'] = int(raw['UDP'].split('length: ')[1].split(' ')[0])
        if 'AODV_Type' in raw:
            p['AODV'] = {}
            p['AODV']['type'] = raw['AODV_Type']
        if 'AODV_RREP' in raw:
            p['AODV']['RREP'] = {}
            p['AODV']['RREP']['dst'] = raw['AODV_RREP'].split('destination: ipv4 ')[1].split(' ')[0]
            p['AODV']['RREP']['src'] = raw['AODV_RREP'].split('source ipv4 ')[1].split(' ')[0]
            p['AODV']['RREP']['SN'] = int(raw['AODV_RREP'].split('sequence number ')[1].split(' ')[0])
            p['AODV']['RREP']['lifetime'] = int(raw['AODV_RREP'].split('lifetime ')[1].split(' ')[0])
            p['AODV']['RREP']['acknowledgment'] = raw['AODV_RREP'].split('acknowledgment ')[1].split(' ')[0]
            p['AODV']['RREP']['flag'] = raw['AODV_RREP'].split('flag ')[1]
        if 'AODV_RREQ' in raw:
            p['AODV']['RREQ'] = {}
            p['AODV']['RREQ']['ID'] = int(raw['AODV_RREQ'].split('ID ')[1].split(' ')[0])
            p['AODV']['RREQ']['dst'] = raw['AODV_RREQ'].split('destination: ipv4 ')[1].split(' ')[0]
            p['AODV']['RREQ']['src'] = raw['AODV_RREQ'].split('source: ipv4 ')[1].split(' ')[0]
            p['AODV']['RREQ']['SN'] = int(raw['AODV_RREQ'].split('sequence number ')[1].split(' ')[0])
            p['AODV']['RREQ']['flags'] = raw['AODV_RREQ'].split('flags: ')[1]
        if 'AODV_RACK' in raw:
            pass # seems to be empty
        if 'AODV_RERR' in raw:
            p['AODV']['RERR'] = raw['AODV_RERR']
        if 'ICMPv4_HEADER' in raw:
            p['ICMPv4'] = {}
            p['ICMPv4']['type'] = int(raw['ICMPv4_HEADER'].split('type=')[1].split(',')[0])
            p['ICMPv4']['code'] = int(raw['ICMPv4_HEADER'].split('code=')[1])
        if 'ICMPv4_TE' in raw:
            p['ICMPv4']['data'] = raw['ICMPv4_TE']
        if 'ICMPv4_DU' in raw:
            p['ICMPv4']['data'] = raw['ICMPv4_DU']

        if len(p) > 0:
            packets.append(p)
        return packets

In [397]:
path = "../data/wormhole"
rng = 100
routes = list()
flows = list()
packets = list()

for i in range(rng):

    for routes_file in glob.glob('{}/output_{}/*.routes'.format(path, i)):

        info = re.findall('(\w+)/output_(\d+)', routes_file)

        handle = open(routes_file, 'r')
        data = handle.read()
        handle.close()

        nodes = re.split('\n\n', data)
        nodes.pop()

        for node in nodes:

            header = re.findall('Node:\s+(\d+)\s+Time:\s+(\d+)', node)
            lines = re.findall('(\d{1,3}(?:\.\d{1,3}){3})\s+(\d{1,3}(?:\.\d{1,3}){3})\s+(\d{1,3}(?:\.\d{1,3}){3})\s+(\w+)\s+(-?\d+\.\d+)\s+(\d+)', node)

            for line in lines:
                l = list(line)
                l[4] = float(line[4])
                l[5] = int(line[5])
                routes.append(header[0] + tuple(l) + info[0])

    for flowmon_file in glob.glob('{}/output_{}/*.flowmon'.format(path, i)):

        info = re.findall('(\w+)/output_(\d+)', flowmon_file)

        with open(flowmon_file) as fobj:
            xml = fobj.read()

        root = etree.fromstring(xml)


        for flow in root.xpath('/FlowMonitor/FlowStats/Flow'):

            attributes = list()

            for attrib in flow.attrib:

                attr = flow.attrib[attrib]
                if 'ns' in attr:
                    attr = re.findall('(\d+)', attr)[0]

                attributes.append(int(attr))

            flows.append(tuple(attributes) + info[0])
    
    for packets_file in glob.glob('{}/output_{}/*.xml'.format(path, i)):
        
        if "routingtable-wireless.xml" in packets_file:
            continue

        info = re.findall('(\w+)/output_(\d+)', packets_file)
        
        packets_dict = parce_packets(packets_file)
        table_packets = []
        for p in packets_dict:
            if 'IPv4' in p:
                table_packets.append([mac_to_id(p['MAC']['src']), mac_to_id(p['MAC']['dst']), ip_to_id(p['IPv4']['src']), ip_to_id(p['IPv4']['dst'])])

        table_packets = pd.DataFrame(table, columns=['mac_src', 'mac_dst', 'ip_src', 'ip_dst'])
        table_packets = table_packets[table_packets['ip_dst'] != 254]
        table_packets['input'] = 0
        table_packets['output'] = 0
        input_cnt = table_packets.groupby('mac_src').agg({'input':'count'})
        output_cnt = table_packets.groupby('mac_dst').agg({'output':'count'})
        io_mac = input_cnt.join(output_cnt)
        io_mac["diff"] = io_mac['input'] - io_mac['output']
        io_mac["normalized"] = io_mac["diff"] - io_mac["diff"].min() + 1

        input = io_mac.agg({'input': [entropy, 'min', 'max', 'mean', 'var']}).values.flatten()
        output = io_mac.agg({'output': [entropy, 'min', 'max', 'mean', 'var']}).values.flatten()
        diff = io_mac.agg({'diff': ['min', 'max', 'mean', 'var']}).values.flatten()
        normalized = io_mac.agg({'normalized': [entropy, 'min', 'max', 'mean', 'var']}).values.flatten()
        src_count = table[table['mac_src'] == table['ip_src']].count()[0]
        dst_count = table[table['mac_dst'] == table['ip_dst']].count()[0]
        
        packets.append(np.concatenate((input, output, diff, normalized, [src_count, dst_count], info[0])))


routes_table = pd.DataFrame(routes, columns=['Node', 'Time', 'Destination', 'Gateway', 'Interface', 'Flag', 'Expire', 'Hops', 'Type', 'Test'])

flag_count = routes_table.groupby(['Type', 'Test', 'Time', 'Flag']).agg({'Flag' : ['count']})
flag_count = flag_count.reset_index(col_level=1)
flag_count.columns = flag_count.columns.droplevel()
flag_agg = flag_count.groupby(['Type', 'Test', 'Flag']).agg({'count': ['max', 'min', 'mean', 'var']}).unstack().reset_index()
flag_agg.columns = [col[0]+col[1]+col[2] for col in flag_agg.columns]
flag_agg = flag_agg.set_index(['Type', 'Test'])

hops_node_dest_agg = routes_table.groupby(['Type', 'Test', 'Node', 'Destination']).agg({'Hops' : ['min', 'max', 'mean']})
hops_node_agg = hops_node_dest_agg.reset_index().groupby(["Type", "Test", "Node"]).agg(['min', 'max', 'mean']).reset_index(col_level=1)
hops_node_agg.columns = hops_node_agg.columns.droplevel()
hops_agg = hops_node_agg.reset_index().groupby(["Type", "Test"]).agg(['min', 'max', 'mean']).reset_index(col_level=1)
hops_agg.columns = [col[0]+col[1]+col[2] for col in hops_agg.columns]
hops_agg = hops_agg.set_index(['Type', 'Test'])

flows_table = pd.DataFrame(flows, columns=['flowId', 'timeFirstTxPacket', 'timeFirstRxPacket', 'timeLastTxPacket', 'timeLastRxPacket', 'delaySum', 'jitterSum', 'lastDelay', 'txBytes', 'rxBytes', 'txPackets', 'rxPackets', 'lostPackets', 'timesForwarded', 'Type', 'Test'])

lost_agg = flows_table.groupby(['Type', 'Test']).agg({'lostPackets' : ['sum', 'mean']})
forwarded_agg = flows_table.groupby(['Type', 'Test']).agg({'timesForwarded' : ['sum', 'max', 'mean', 'var']})

packets_table = pd.DataFrame(packets, columns=["entopy_input", "min_input", "max_input", "mean_input", "var_input", "entopy_output", "min_output", "max_output", "mean_output", "var_output", "min_diff", "max_diff", "mean_diff", "var_diff", "entopy_normolized", "min_normolized", "max_normolized", "mean_normolized", "var_normolized", "src_count", "dst_count", 'Type', 'Test'])
packets_table = packets_table.set_index(['Type', 'Test'])

#return flag_agg.join(hops_agg).join(lost_agg).join(forwarded_agg).join(packets_table)

/home/nsuprotivniy/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:2530: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [387]:
result.shape

(100, 69)

In [398]:
flag_count = routes_table.groupby(['Type', 'Test', 'Time', 'Flag']).agg({'Flag' : ['count']})
flag_count = flag_count.reset_index(col_level=1)
flag_count.columns = flag_count.columns.droplevel()
flag_agg = flag_count.groupby(['Type', 'Test', 'Flag']).agg({'count': ['max', 'min', 'mean', 'var']}).unstack().reset_index()
flag_agg.columns = [col[0]+col[1]+col[2] for col in flag_agg.columns]
flag_agg = flag_agg.set_index(['Type', 'Test'])

In [456]:
tmp = flag_count.groupby(['Type', 'Test', 'Flag']).agg({'count': ['max', 'min', 'mean', 'var']})

In [461]:
tmp.index.set_levels(['DOWN', 'UP', 'IN_SEARCH'], level=2, inplace=True)

In [463]:
tmp

count                   
                     max min       mean var
Type     Test Flag                         
wormhole 0    DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         1    DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         10   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         11   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         12   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         13   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         14   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         15   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         16   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         17   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         18   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         19   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         2    DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         20   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         21   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
...                  ...  ..        ...  ..
         86   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         87   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         88   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         89   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         9    DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         90   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         91   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         92   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         93   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         94   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         95   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         96   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         97   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         98   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1
         99   DOWN     2   2   2.000000   0
              UP      56  54  54.666667   1

[200 rows x 4 columns]

In [424]:
tmp = flag_count.groupby(['Type', 'Test', 'Flag']).agg({'count': ['max', 'min', 'mean', 'var']}).unstack()

In [444]:
flag_count.groupby(['Type', 'Test', 'Flag']).agg({'count': ['max', 'min', 'mean', 'var']}).unstack().columns.get_level_values(2)

Index(['DOWN', 'UP', 'DOWN', 'UP', 'DOWN', 'UP', 'DOWN', 'UP'], dtype='object', name='Flag')

In [425]:
tmp

count                                     
                max      min     mean             var   
Flag           DOWN  UP DOWN  UP DOWN         UP DOWN UP
Type     Test                                           
wormhole 0        2  56    2  54  2.0  54.666667    0  1
         1        2  56    2  54  2.0  54.666667    0  1
         10       2  56    2  54  2.0  54.666667    0  1
         11       2  56    2  54  2.0  54.666667    0  1
         12       2  56    2  54  2.0  54.666667    0  1
         13       2  56    2  54  2.0  54.666667    0  1
         14       2  56    2  54  2.0  54.666667    0  1
         15       2  56    2  54  2.0  54.666667    0  1
         16       2  56    2  54  2.0  54.666667    0  1
         17       2  56    2  54  2.0  54.666667    0  1
         18       2  56    2  54  2.0  54.666667    0  1
         19       2  56    2  54  2.0  54.666667    0  1
         2        2  56    2  54  2.0  54.666667    0  1
         20       2  56    2  54  2.0  54.666667    0  1
         21       2  56    2  54  2.0  54.666667    0  1
         22       2  56    2  54  2.0  54.666667    0  1
         23       2  56    2  54  2.0  54.666667    0  1
         24       2  56    2  54  2.0  54.666667    0  1
         25       2  56    2  54  2.0  54.666667    0  1
         26       2  56    2  54  2.0  54.666667    0  1
         27       2  56    2  54  2.0  54.666667    0  1
         28       2  56    2  54  2.0  54.666667    0  1
         29       2  56    2  54  2.0  54.666667    0  1
         3        2  56    2  54  2.0  54.666667    0  1
         30       2  56    2  54  2.0  54.666667    0  1
         31       2  56    2  54  2.0  54.666667    0  1
         32       2  56    2  54  2.0  54.666667    0  1
         33       2  56    2  54  2.0  54.666667    0  1
         34       2  56    2  54  2.0  54.666667    0  1
         35       2  56    2  54  2.0  54.666667    0  1
...             ...  ..  ...  ..  ...        ...  ... ..
         72       2  56    2  54  2.0  54.666667    0  1
         73       2  56    2  54  2.0  54.666667    0  1
         74       2  56    2  54  2.0  54.666667    0  1
         75       2  56    2  54  2.0  54.666667    0  1
         76       2  56    2  54  2.0  54.666667    0  1
         77       2  56    2  54  2.0  54.666667    0  1
         78       2  56    2  54  2.0  54.666667    0  1
         79       2  56    2  54  2.0  54.666667    0  1
         8        2  56    2  54  2.0  54.666667    0  1
         80       2  56    2  54  2.0  54.666667    0  1
         81       2  56    2  54  2.0  54.666667    0  1
         82       2  56    2  54  2.0  54.666667    0  1
         83       2  56    2  54  2.0  54.666667    0  1
         84       2  56    2  54  2.0  54.666667    0  1
         85       2  56    2  54  2.0  54.666667    0  1
         86       2  56    2  54  2.0  54.666667    0  1
         87       2  56    2  54  2.0  54.666667    0  1
         88       2  56    2  54  2.0  54.666667    0  1
         89       2  56    2  54  2.0  54.666667    0  1
         9        2  56    2  54  2.0  54.666667    0  1
         90       2  56    2  54  2.0  54.666667    0  1
         91       2  56    2  54  2.0  54.666667    0  1
         92       2  56    2  54  2.0  54.666667    0  1
         93       2  56    2  54  2.0  54.666667    0  1
         94       2  56    2  54  2.0  54.666667    0  1
         95       2  56    2  54  2.0  54.666667    0  1
         96       2  56    2  54  2.0  54.666667    0  1
         97       2  56    2  54  2.0  54.666667    0  1
         98       2  56    2  54  2.0  54.666667    0  1
         99       2  56    2  54  2.0  54.666667    0  1

[100 rows x 8 columns]

In [349]:
packets_table.shape

(100, 23)

In [323]:
flag_agg

Type Test count                                               \
                       max                 min                     mean   
Flag                  DOWN IN_SEARCH   UP DOWN IN_SEARCH  UP       DOWN   
0     malicious    0    53         9  157    1         7  60  37.000000   
1     malicious    1    54         6  161   14         6  60  39.600000   
2     malicious   10    47         9  147    9         8  60  32.450000   
3     malicious   11    60         8  164    2         1  60  41.952381   
4     malicious   12    56         9  168   13         8  60  42.100000   
5     malicious   13    30        10  138    1         2  60  19.904762   
6     malicious   14    33         9  137    3         1  60  21.095238   
7     malicious   15    58         7  168   15         7  60  44.450000   
8     malicious   16    32         5  137    1         5  60  21.380952   
9     malicious   17    49         8  152   16         8  60  35.700000   
10    malicious   18    49         7  159    2         1  60  35.952381   
11    malicious   19    64        10  170   21         1  60  45.550000   
12    malicious    2    46         8  150   11         1  60  32.545455   
13    malicious   20    43         9  147    1         1  60  29.380952   
14    malicious   21    63         8  166   17         7  60  45.500000   
15    malicious   22    53         7  154   12         7  60  39.300000   
16    malicious   23    41         8  149    1         7  60  28.772727   
17    malicious   24    49         8  156    4         8  60  34.904762   
18    malicious   25    44         8  148    4         7  60  31.333333   
19    malicious   26    45        10  146    3         9  60  30.952381   
20    malicious   27    52         8  161    3         7  60  37.809524   
21    malicious   28    74        10  176   18         8  60  54.000000   
22    malicious   29    47         9  146    2         8  60  31.714286   
23    malicious    3    47         8  147    2         1  60  31.523810   
24    malicious   30    44         8  144    9         7  60  31.250000   
25    malicious   31    36         8  139    1         8  60  25.476190   
26    malicious   32    46         7  153    4         6  60  32.956522   
27    malicious   33    49         9  153    1         8  60  32.090909   
28    malicious   34    47         7  157   12         6  60  35.900000   
29    malicious   35    36         8  139    3         8  60  24.904762   
..          ...  ...   ...       ...  ...  ...       ...  ..        ...   
70    malicious   72    52         8  154    6         1  60  37.476190   
71    malicious   73    41        10  141    1         1  60  26.318182   
72    malicious   74    46         9  147    1         1  60  29.272727   
73    malicious   75    63         8  165   14         7  60  44.150000   
74    malicious   76    48         6  153   13         6  60  34.100000   
75    malicious   77    35         9  137    3         8  60  23.000000   
76    malicious   78    39         8  142    4         8  60  26.666667   
77    malicious   79    51         7  156    4         7  60  36.190476   
78    malicious    8    35         7  137   12         7  60  24.950000   
79    malicious   80    48         7  156    1         7  60  33.500000   
80    malicious   81    33         9  137    3         9  60  22.000000   
81    malicious   82    37         9  142    1         9  60  25.190476   
82    malicious   83    42         8  147   14         8  60  30.550000   
83    malicious   84    45         8  151   12         8  60  33.050000   
84    malicious   85    29         8  137    2         8  60  21.000000   
85    malicious   86    48        11  150    4        10  60  34.761905   
86    malicious   87    46         8  151    5         1  60  32.590909   
87    malicious   88    40        10  146    1        10  60  27.476190   
88    malicious   89    47         8  148   13         8  60  32.950000   
89    malicious    9    52         7

In [324]:
packets_table

,entopy_input,min_input,max_input,mean_input,var_input,entopy_output,min_output,max_output,mean_output,var_output,...,var_diff,entopy_normolized,min_normolized,max_normolized,mean_normolized,var_normolized,src_count,dst_count,Type,Test
0,1.1622999115541472,2.0,212.0,41.2962962962963,2706.831908831909,1.1246536892481531,2.0,237.0,41.2962962962963,3179.6011396011386,...,2853.4615384615386,1.4239652440306785,1.0,317.0,232.0,2853.4615384615386,460,220,malicious,0
1,1.1622999115541472,2.0,212.0,41.2962962962963,2706.831908831909,1.1246536892481531,2.0,237.0,41.2962962962963,3179.6011396011386,...,2853.4615384615386,1.4239652440306785,1.0,317.0,232.0,2853.4615384615386,460,220,malicious,0
2,1.1622999115541472,2.0,212.0,41.2962962962963,2706.831908831909,1.1246536892481531,2.0,237.0,41.2962962962963,3179.6011396011386,...,2853.4615384615386,1.4239652440306785,1.0,317.0,232.0,2853.4615384615386,460,220,malicious,1
3,1.1622999115541472,2.0,212.0,41.2962962962963,2706.831908831909,1.1246536892481531,2.0,237.0,41.2962962962963,3179.6011396011386,...,2853.4615384615386,1.4239652440306785,1.0,317.0,232.0,2853.4615384615386,460,220,malicious,1
4,1.1622999115541472,2.0,212.0,41.2962962962963,2706.831908831909,1.1246536892481531,2.0,237.0,41.2962962962963,3179.6011396011386,...,2853.4615384615386,1.4239652440306785,1.0,317.0,232.0,2853.4615384615386,460,220,malicious,2
5,1.1622999115541472,2.0,212.0,41.2962962962963,2706.831908831909,1.1246536892481531,2.0,237.0,41.2962962962963,3179.6011396011386,...,2853.4615384615386,1.4239652440306785,1.0,317.0,232.0,2853.4615384615386,460,220,malicious,2
6,1.1622999115541472,2.0,212.0,41.2962962962963,2706.831908831909,1.1246536892481531,2.0,237.0,41.2962962962963,3179.6011396011386,...,2853.4615384615386,1.4239652440306785,1.0,317.0,232.0,2853.4615384615386,460,220,malicious,3
7,1.1622999115541472,2.0,212.0,41.2962962962963,2706.831908831909,1.1246536892481531,2.0,237.0,41.2962962962963,3179.6011396011386,...,2853.4615384615386,1.4239652440306785,1.0,317.0,232.0,2853.4615384615386,460,220,malicious,3
8,1.1622999115541472,2.0,212.0,41.2962962962963,2706.831908831909,1.1246536892481531,2.0,237.0,41.2962962962963,3179.6011396011386,...,2853.4615384615386,1.4239652440306785,1.0,317.0,232.0,2853.4615384615386,460,220,malicious,4
9,1.1622999115541472,2.0,212.0,41.2962962962963,2706.831908831909,1.1246536892481531,2.0,237.0,41.2962962962963,3179.6011396011386,...,2853.4615384615386,1.4239652440306785,1.0,317.0,232.0,2853.4615384615386,460,220,malicious,4


In [346]:
packets_table.shape

(100, 23)

In [287]:
hops_node_agg.columns = pd.Index(hops_node_agg.columns)

In [294]:
hops_node_agg

,Type,Test,Node,min,max,mean,min,max,mean,min,max,mean
0,malicious,0,0,1,3,1.333333,1,3,1.333333,1.000000,3.000000,1.333333
1,malicious,0,1,1,2,1.100000,1,2,1.100000,1.000000,2.000000,1.100000
2,malicious,0,10,1,2,1.125000,1,2,1.250000,1.000000,2.000000,1.173913
3,malicious,0,11,0,2,1.000000,0,2,1.000000,0.000000,2.000000,1.000000
4,malicious,0,12,1,1,1.000000,1,1,1.000000,1.000000,1.000000,1.000000
5,malicious,0,13,0,1,0.750000,0,1,0.750000,0.000000,1.000000,0.750000
6,malicious,0,14,0,3,0.916667,0,3,1.000000,0.000000,3.000000,0.991667
7,malicious,0,15,1,3,1.272727,1,3,1.272727,1.000000,3.000000,1.272727
8,malicious,0,16,1,1,1.000000,1,1,1.000000,1.000000,1.000000,1.000000
9,malicious,0,17,1,1,1.000000,1,1,1.000000,1.000000,1.000000,1.000000


In [237]:
flag_d_agg.columns = ['Test', 'Type',
                     'DOWN_max', 'IN_SEARCH_max', 'UP_max',\
                     'DOWN_min', 'IN_SEARCH_min', 'UP_min',\
                     'DOWN_mean', 'IN_SEARCH_mean', 'UP_mean'\
                     'DOWN_', 'IN_SEARCH_mean', 'UP_mean'\]

MultiIndex(levels=[['count', 'Test', 'Type'], ['max', 'min', 'mean', 'var', ''], ['DOWN', 'IN_SEARCH', 'UP', '']],
           labels=[[2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [4, 4, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3], [3, 3, 0, 1, 2, 0, 1, 2, 0, 1, 2, 0, 1, 2]],
           names=[None, None, 'Flag'])